# NLP - SPAM detection

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('spam').getOrCreate()

In [6]:
data =spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema = True, sep='\t')

In [7]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [10]:
#Rename the columns
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [11]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import length

In [13]:
data = data.withColumn('length', length(data['text']))

In [14]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [16]:
data.groupBy('class').mean().show() # see that spam messages are longer

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [17]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [26]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [21]:
from pyspark.ml.classification import NaiveBayes

In [22]:
nb = NaiveBayes()

In [23]:
from pyspark.ml import Pipeline

In [27]:
data_prep_line = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [28]:
cleaner = data_prep_line.fit(data)

In [29]:
clean_data = cleaner.transform(data)

In [32]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [34]:
clean_data=clean_data.select('label','features')

In [35]:
(train,test) = clean_data.randomSplit([0.7, 0.3])

In [38]:
spam_detector = nb.fit(train)

21/11/30 13:18:51 WARN DAGScheduler: Broadcasting large task binary with size 1142.6 KiB
21/11/30 13:18:52 WARN DAGScheduler: Broadcasting large task binary with size 1126.2 KiB


In [40]:
test_results = spam_detector.transform(test)

In [41]:
test_results.show()

21/11/30 13:19:37 WARN DAGScheduler: Broadcasting large task binary with size 1360.0 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,4,50,...|[-827.63321464391...|[1.0,1.6318464888...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1002.5565227035...|[1.0,2.6454190913...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-655.84673103672...|[1.0,1.4121392876...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-804.74916846464...|[1.0,2.4042776281...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-848.36864722348...|[1.0,4.1807244429...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1296.9567908256...|[1.0,1.1317938263...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1536.9507822561...|[0.99997808637738...|       0.0|
|  0.0|(13424,[0,1,30,12...|[-598.73276309133...|[1.0,4.1154023758...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-250.67482247721...|[0.87493718314227...|       0.0|
|  0.0|(13424,[0

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
acc_eval = MulticlassClassificationEvaluator()

In [44]:
acc = acc_eval.evaluate(test_results)

21/11/30 13:20:20 WARN DAGScheduler: Broadcasting large task binary with size 1365.1 KiB


In [45]:
print('Accuracy of Naive Bayes')
print(acc)

Accuracy of Naive Bayes
0.9205272799918418
